In [ ]:
import numpy as np
import pandas as pd

# Вспомогательный класс: Решающий пень (Decision Stump)
class DecisionStump:
    """
    Простой решающий пень для бинарной классификации.
    Классифицирует на основе порогового значения одного признака.
    """
    def __init__(self):
        self.feature_index = None # Индекс признака, по которому происходит разделение
        self.threshold = None     # Пороговое значение для разделения
        self.polarity = 1         # Направление неравенства (1 или -1)
        self.alpha = None         # Вес этого пня в финальном ансамбле

    def predict(self, X):
        """
        Прогнозирует метки классов для входных данных.

        Args:
            X (np.ndarray или pd.DataFrame): Входные данные.

        Returns:
            np.ndarray: Спрогнозированные метки классов (-1 или 1).
        """
        X = np.asarray(X)
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)

        # Применяем правило пня: если значение признака <= порога, то метка polarity, иначе -polarity
        if self.polarity == 1:
            predictions[X[:, self.feature_index] <= self.threshold] = -1
        else: # polarity == -1
            predictions[X[:, self.feature_index] > self.threshold] = -1

        return predictions

# Реализация алгоритма AdaBoost
class SimpleAdaBoost:
    """
    Простая реализация алгоритма AdaBoost для бинарной классификации
    с использованием решающих пней в качестве слабых классификаторов.
    Использует только numpy и pandas.
    """
    def __init__(self, n_estimators=50):
        """
        Инициализация AdaBoost.

        Args:
            n_estimators (int): Количество слабых классификаторов (решающих пней). По умолчанию 50.
        """
        self.n_estimators = n_estimators
        self.estimators = [] # Список слабых классификаторов (DecisionStump)
        self.estimator_weights = [] # Веса каждого классификатора (alpha)

    def fit(self, X, y):
        """
        Обучает модель AdaBoost на данных.

        Args:
            X (np.ndarray или pd.DataFrame): Входные признаки.
            y (np.ndarray или pd.Series): Метки классов (-1 или 1).
        """
        X = np.asarray(X)
        y = np.asarray(y)

        n_samples, n_features = X.shape

        # Проверяем, что метки классов -1 или 1
        if not np.all(np.isin(y, [-1, 1])):
             raise ValueError("Метки классов должны быть -1 или 1")

        # Инициализируем веса примеров данных равномерно
        sample_weights = np.full(n_samples, (1 / n_samples))

        self.estimators = []
        self.estimator_weights = []

        # Основной цикл AdaBoost
        for _ in range(self.n_estimators):
            # Находим лучший решающий пень для текущих весов примеров
            stump = self._find_best_stump(X, y, sample_weights)

            # Вычисляем ошибку классификации пня
            predictions = stump.predict(X)
            misclassified = (predictions != y)
            # Взвешенная ошибка: сумма весов неправильно классифицированных примеров
            error = np.sum(sample_weights[misclassified])

            # Избегаем деления на ноль или логарифмирования нуля
            if error > 0.5 or error < 1e-9:
                 # Если ошибка > 0.5, пень хуже случайного угадывания. Останавливаемся.
                 # Если ошибка очень близка к 0, пень идеален. Присваиваем ему большой вес.
                 alpha = 1000.0 if error < 1e-9 else 0.0
            else:
                # Вычисляем вес пня (alpha)
                alpha = 0.5 * np.log((1.0 - error) / (error + 1e-10)) # Добавляем эпсилон для стабильности

            # Обновляем веса примеров
            # Увеличиваем веса неправильно классифицированных примеров
            # Уменьшаем веса правильно классифицированных примеров
            # new_weight = old_weight * exp(-alpha * y * prediction)
            sample_weights *= np.exp(-alpha * y * predictions)

            # Нормализуем веса примеров, чтобы их сумма была равна 1
            sample_weights /= np.sum(sample_weights)

            # Сохраняем пень и его вес
            stump.alpha = alpha
            self.estimators.append(stump)
            self.estimator_weights.append(alpha)

    def _find_best_stump(self, X, y, sample_weights):
        """
        Находит лучший решающий пень, минимизирующий взвешенную ошибку.

        Args:
            X (np.ndarray): Входные признаки.
            y (np.ndarray): Метки классов (-1 или 1).
            sample_weights (np.ndarray): Веса каждого примера.

        Returns:
            DecisionStump: Лучший найденный решающий пень.
        """
        n_samples, n_features = X.shape
        best_stump = None
        min_error = float('inf')

        # Перебираем все признаки
        for feature_index in range(n_features):
            # Получаем уникальные значения признака, чтобы использовать их как потенциальные пороги
            feature_values = X[:, feature_index]
            unique_values = np.unique(feature_values)

            # Перебираем все уникальные значения как потенциальные пороги
            # Пороги обычно выбирают между уникальными значениями
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2 if len(unique_values) > 1 else unique_values

            if len(thresholds) == 0:
                 # Если только одно уникальное значение, используем его как порог
                 thresholds = unique_values

            for threshold in thresholds:
                # Проверяем два варианта полярности (направления неравенства)
                for polarity in [1, -1]:
                    stump = DecisionStump()
                    stump.feature_index = feature_index
                    stump.threshold = threshold
                    stump.polarity = polarity

                    # Вычисляем ошибку для текущего пня
                    predictions = stump.predict(X)
                    misclassified = (predictions != y)
                    error = np.sum(sample_weights[misclassified])

                    # Если текущая ошибка меньше минимальной, обновляем лучший пень
                    if error < min_error:
                        min_error = error
                        best_stump = stump

        return best_stump

    def predict(self, X):
        """
        Прогнозирует метки классов для входных данных, используя ансамбль пней.

        Args:
            X (np.ndarray или pd.DataFrame): Входные данные.

        Returns:
            np.ndarray: Спрогнозированные метки классов (-1 или 1).
        """
        X = np.asarray(X)
        n_samples = X.shape[0]
        # Сумма взвешенных прогнозов от всех пней
        weighted_predictions = np.zeros(n_samples)

        for stump, alpha in zip(self.estimators, self.estimator_weights):
            weighted_predictions += alpha * stump.predict(X)

        # Финальное решение: знак суммы взвешенных прогнозов
        # Используем signbit для корректной обработки нуля
        predictions = np.where(np.signbit(weighted_predictions), -1, 1)

        return predictions

# 2. Симуляция данных для тестирования AdaBoost

# Генерируем простые двумерные данные для бинарной классификации
np.random.seed(42)
n_samples_sim = 100

# Класс -1: точки вокруг (-1, -1)
X_neg = np.random.multivariate_normal([-1, -1], [[0.5, 0], [0, 0.5]], n_samples_sim // 2)
y_neg = np.full(n_samples_sim // 2, -1)

# Класс 1: точки вокруг (1, 1)
X_pos = np.random.multivariate_normal([1, 1], [[0.5, 0], [0, 0.5]], n_samples_sim // 2)
y_pos = np.full(n_samples_sim // 2, 1)

X_sim = np.vstack((X_neg, X_pos))
y_sim = np.hstack((y_neg, y_pos))

# Перемешиваем данные
shuffle_indices = np.random.permutation(n_samples_sim)
X_sim = X_sim[shuffle_indices]
y_sim = y_sim[shuffle_indices]

# 3. Тестирование и сравнение

# Используем нашу реализацию AdaBoost
my_adaboost = SimpleAdaBoost(n_estimators=10) # Используем 10 пней для примера
my_adaboost.fit(X_sim, y_sim)
my_predictions = my_adaboost.predict(X_sim)

# Вычисляем точность нашей модели
my_accuracy = np.mean(my_predictions == y_sim)
print(f"Точность нашей реализации AdaBoost на симулированных данных: {my_accuracy:.4f}")

# Сравнение со стандартной реализацией (sklearn)
# Для сравнения разрешено использовать sklearn
try:
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import accuracy_score

    # Используем AdaBoostClassifier из sklearn с решающими пнями (max_depth=1)
    # random_state для воспроизводимости
    sklearn_adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                                          n_estimators=10, random_state=42, algorithm='SAMME') # SAMME для дискретных прогнозов
    sklearn_adaboost.fit(X_sim, y_sim)
    sklearn_predictions = sklearn_adaboost.predict(X_sim)

    # Вычисляем точность sklearn
    sklearn_accuracy = accuracy_score(y_sim, sklearn_predictions)
    print(f"Точность sklearn AdaBoost на симулированных данных: {sklearn_accuracy:.4f}")

    # Сравнение предсказаний (могут немного отличаться из-за деталей реализации)
    agreement = np.mean(my_predictions == sklearn_predictions)
    print(f"Доля совпадений предсказаний нашей и sklearn реализаций: {agreement:.4f}")

except ImportError:
    print("\nБиблиотека sklearn не найдена. Пропуск сравнения.")
except Exception as e:
    print(f"\nОшибка при сравнении со sklearn: {e}")

# Визуализация границы решений (опционально, требует matplotlib)
# import matplotlib.pyplot as plt
#
# def plot_decision_boundary(predict_func, X, y, title):
#     x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
#     y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
#     xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
#                          np.arange(y_min, y_max, 0.1))
#
#     Z = predict_func(np.c_[xx.ravel(), yy.ravel()])
#     Z = Z.reshape(xx.shape)
#
#     plt.contourf(xx, yy, Z, alpha=0.4)
#     plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')
#     plt.title(title)
#     plt.xlabel('Признак 1')
#     plt.ylabel('Признак 2')
#
# plt.figure(figsize=(12, 5))
#
# plt.subplot(1, 2, 1)
# plot_decision_boundary(my_adaboost.predict, X_sim, y_sim, 'Наша реализация AdaBoost')
#
# if 'sklearn_adaboost' in locals():
#     plt.subplot(1, 2, 2)
#     plot_decision_boundary(sklearn_adaboost.predict, X_sim, y_sim, 'sklearn AdaBoost')
#
# plt.tight_layout()
# plt.show()

